In [1]:
import os
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [2]:
assistant_id = "asst_si2Szhtw1XK8mulXeE8UhEzl"

assistant = client.beta.assistants.retrieve(assistant_id)
print(assistant)

Assistant(id='asst_si2Szhtw1XK8mulXeE8UhEzl', created_at=1710963271, description=None, file_ids=[], instructions='You are a helpful AI assistant that provides information about a company such as earnings call transcripts, stock information such as current price, 52 week high and low price, and company information such as industry, summary.You get all your information from the fuctions available to you only. You only answer questions based on the information you have been given. You have access to a sandboxed environment for writing and testing code.If need need clarifications please ask the user to provide more information.You are given various functions that provide all the data you need to answer questions.You might have to call multiple functions to get the all needed information', metadata={}, model='gpt-35-turbo', name='Company-Insights', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter'), ToolFunction(function=FunctionDefinition(name='get_earnings_call_transc

In [3]:
thread = client.beta.threads.create()
#print(thread)

In [4]:
# Add a user question to the thread
msg = "what are strategic priorities for the year for JNJ"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=msg
)

thread_messages = client.beta.threads.messages.list(thread.id)

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [10]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

requires_action


In [7]:
from functions.company import get_company_info

import time

max_steps = 10 
sleep = .5

#add the available functions here   
available_functions = {"get_company_info": get_company_info}

try:
    cnt = 0
    while cnt < max_steps:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)        
        cnt += 1
        if run.status == "requires_action":
            tool_responses = []
            if (
                run.required_action.type == "submit_tool_outputs"
                and run.required_action.submit_tool_outputs.tool_calls is not None
            ):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls

                for call in tool_calls:
                    if call.type == "function":
                        if call.function.name not in available_functions:
                            raise Exception("Function requested by the model does not exist")
                        function_to_call = available_functions[call.function.name]
                        print(call.function.arguments)
                        print(call.function.name)
                        tool_response = function_to_call(**json.loads(call.function.arguments))
                        tool_responses.append({"tool_call_id": call.id, "output": tool_response})

            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
            )

            print(run.status)

        if run.status == "failed":
            print("Run failed.")
            break
        if run.status == "completed":
            break
        time.sleep(sleep)

except Exception as e:
    print(e)



{
  "question": "strategic priorities",
  "company_stock_ticker": "JNJ"
}
get_company_info
Error code: 400 - {'error': {'message': '1 validation error for Request\nbody -> tool_outputs -> 0 -> output\n  none is not an allowed value (type=type_error.none.not_allowed)', 'type': 'invalid_request_error', 'param': None, 'code': None}}


In [14]:
import os
import requests
import logging

# Create a custom logger
logger = logging.getLogger(__name__)
# Create a file handler
#handler = logging.FileHandler('app.log')
# Add the handler to the logger
#logger.addHandler(handler)

def get_company_info(company_stock_ticker: str, question: str): 
    api_key = "0slxspiAvFd9w6xrvw1RTcAP1wnd13QH"        
    url = "https://hackathon-ai-search-pf.canadaeast.inference.ml.azure.com/score"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}',
        'azureml-model-deployment': 'hackathon-ai-search-pf-1'
    }       
    payload = {
        'client': company_stock_ticker,
        'question': question
    }

    logger.info(f"Getting company information at {url}")
    response = requests.post(url, json=payload, headers=headers)

    print(response.status_code)

    if response.status_code == 200:
        data = response.json()
        logger.info(f"Company info for {company_stock_ticker} found: {data}")
        return data
    else:
        logger.info(f"Company info for {company_stock_ticker} not found") 
        return None 

r = get_company_info("JNJ", "what are strategic priorities for the year for JNJ")
print(r) 

424
None


In [8]:
print(tool_calls)

[RequiredActionFunctionToolCall(id='call_4RtmOsvgApammm3fNdzAEx0g', function=Function(arguments='{\n  "question": "strategic priorities",\n  "company_stock_ticker": "JNJ"\n}', name='get_company_info'), type='function')]


In [9]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))
print(run.status)

{
  "data": [
    {
      "id": "msg_z4K93wpF5Av3OyDQ4dNIyccv",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "what are strategic priorities for the year for JNJ"
          },
          "type": "text"
        }
      ],
      "created_at": 1710963616,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_H8zAAmu38zbT12syHuXMhgdT"
    }
  ],
  "object": "list",
  "first_id": "msg_z4K93wpF5Av3OyDQ4dNIyccv",
  "last_id": "msg_z4K93wpF5Av3OyDQ4dNIyccv",
  "has_more": false
}
requires_action
